In [1]:
import importlib
from libraries import *
from funtions import Position, BacktestingCapCOM , OptunaOpt , get_portfolio_value
import Objetive
importlib.reload(Objetive)
from Objetive import hyperparams
from dataclasses import dataclass
from Indicadores import get_rsi, get_momentum, get_volatility

data = pd.read_csv('Binance_BTCUSDT_1h.csv', skiprows=1).dropna()
data['Date'] = pd.to_datetime(data['Date'], format='mixed')
data.head()

c:\Users\ppmel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1758582000000,2025-09-22 23:00:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.31095,1.524494e+07,34083
1,1758578400000,2025-09-22 22:00:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.60715,3.264691e+07,42836
2,1758574800000,2025-09-22 21:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.31156,3.307493e+07,42931
3,1758571200000,2025-09-22 20:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.84005,6.707508e+07,93553
4,1758567600000,2025-09-22 19:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.37365,1.467768e+08,126232


In [2]:
pd.DataFrame(data.isnull().sum()).T

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,0,0,0,0,0,0,0,0,0,0


In [9]:
def backtest(data, capital, trial) -> float:

    # --- Hiperparámetros ---
    data = data.copy()  

    params = hyperparams(trial)

    rsi_window = params["rsi_window"]
    rsi_lower = params["rsi_lower"]
    rsi_upper = params["rsi_upper"]
    momentum_window = params["momentum_window"]
    momentum_threshold = params["momentum_threshold"]
    volatility_window = params["volatility_window"]
    volatility_threshold = params["volatility_threshold"]
    stop_loss = params["stop_loss"]
    take_profit = params["take_profit"]
    n_shares = params["n_shares"]

    # --- Señales ---
    buy_rsi, sell_rsi = get_rsi(data, rsi_window, rsi_upper, rsi_lower)
    buy_momentum, sell_momentum = get_momentum(data, momentum_window, momentum_threshold)
    buy_volatility, sell_volatility = get_volatility(data, volatility_window, volatility_threshold)

    data["buy_signal"]  = buy_rsi & buy_momentum & buy_volatility
    data["sell_signal"] = sell_rsi & sell_momentum & sell_volatility

    # --- Backtest ---
    cash = float(capital)
    active_long_positions: list[Position] = []
    active_short_positions: list[Position] = []

    COM = BacktestingCapCOM.COM
    max_positions_per_side = 1

    portfolio_values = [BacktestingCapCOM.initial_capital]

    for i, row in data.iterrows():
        price = row.Close

        # --- Cierre LONG ---
        for pos in active_long_positions.copy():
            if price >= pos.tp or price <= pos.sl:
                cash += price * pos.n_shares * (1 - COM)
                active_long_positions.remove(pos)

        # --- Cierre SHORT ---
        for pos in active_short_positions.copy():
            if price <= pos.tp or price >= pos.sl:
                cash -= price * pos.n_shares * (1 + COM)
                active_short_positions.remove(pos)

        # --- Entrada LONG ---
        if row.buy_signal:
            if len(active_long_positions) < max_positions_per_side:
                cost = price * n_shares * (1 + COM)
                if cash >= cost:
                    cash -= cost
                    pos = Position(
                        ticker="BTCUSDT", n_shares=n_shares, price=price,
                        sl=price * (1 - stop_loss), tp=price * (1 + take_profit),
                        time=getattr(row, "Datetime", None), side="long"
                    )
                    active_long_positions.append(pos)

        # --- Entrada SHORT ---
        if row.sell_signal:
            if len(active_short_positions) < max_positions_per_side:
                entry_cash = price * n_shares * (1 - COM)
                cash += entry_cash
                pos = Position(
                    ticker="BTCUSDT", n_shares=n_shares, price=price,
                    sl=price * (1 + stop_loss), tp=price * (1 - take_profit),
                    time=getattr(row, "Datetime", None), side="short"
                )
                active_short_positions.append(pos)

        # --- Valor del portafolio ---
        unrealized_long  = sum((price - p.price) * p.n_shares for p in active_long_positions)
        unrealized_short = sum((p.price - price) * p.n_shares for p in active_short_positions)
        total_value = cash + unrealized_long + unrealized_short
        portfolio_values.append(total_value)

    # --- Cierre al último precio ---
    final_price = data.iloc[-1].Close
    for pos in active_long_positions:
        cash += final_price * pos.n_shares * (1 - COM)
    for pos in active_short_positions:
        cash -= final_price * pos.n_shares * (1 + COM)

    final_portfolio_value = cash
    retorno_relativo = float(final_portfolio_value / BacktestingCapCOM.initial_capital - 1)

    return retorno_relativo, portfolio_values


In [4]:
capital = 1_000_000
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: backtest(data, capital, trial), n_trials=50, n_jobs=-1)


[I 2025-09-30 23:52:55,161] A new study created in memory with name: no-name-9d5d6659-c1b9-46aa-a211-4299c58f765f
[I 2025-09-30 23:53:33,002] Trial 0 finished with value: 0.0 and parameters: {'rsi_window': 16, 'rsi_lower': 19, 'rsi_upper': 74, 'momentum_window': 46, 'momentum_threshold': 0.6430453091399915, 'volatility_window': 9, 'volatility_threshold': 0.3338780635216624, 'stop_loss': 0.08046110577684759, 'take_profit': 0.13730529648985523, 'n_shares': 3}. Best is trial 0 with value: 0.0.
[I 2025-09-30 23:53:33,684] Trial 4 finished with value: -0.08524487862500008 and parameters: {'rsi_window': 17, 'rsi_lower': 14, 'rsi_upper': 67, 'momentum_window': 49, 'momentum_threshold': 1.1013051201717954, 'volatility_window': 12, 'volatility_threshold': 0.12465820590924373, 'stop_loss': 0.09448464152673174, 'take_profit': 0.020658878194807204, 'n_shares': 10}. Best is trial 0 with value: 0.0.
[I 2025-09-30 23:53:34,194] Trial 3 finished with value: 0.0 and parameters: {'rsi_window': 47, 'rsi_

In [5]:
study.best_params

{'rsi_window': 6,
 'rsi_lower': 13,
 'rsi_upper': 70,
 'momentum_window': 36,
 'momentum_threshold': 0.5875860702531265,
 'volatility_window': 22,
 'volatility_threshold': 0.4998114468999385,
 'stop_loss': 0.03394694217151695,
 'take_profit': 0.07871793817819059,
 'n_shares': 7}

In [6]:
study.best_value

0.5545143233875005